# Sparse Matrix Application: Conjugate Gradient for Solving Linear Systems

Welcome to the first homework for the high-performance computation of sparse matrices! In this work, you'll try to solve symmetric postive-definite sparse linear systems with Conjugate Gradient method.

By the end of this homework, you are expected to be able to:

- Understand the advantage of compressed storage format of sparse matrix over traditional 2D storage format for Sparse Matrix Vector product operation (SpMV). 
- Understand the methology to solve linear system by iterative methods, especially Conjugate Gradient (CG).
- have a basic knowledge on the preconditioning techniques for iterative methods.

## Table of Contents
- [1 - Packages](#1)
- [2 - Test Matrix Collection](#2)
- [3 - Basic Conjugate Gradient Method](#3)
- [4 - Precondtioned Conjugate Gradient Method](#4)

<a name='1'></a>
## 1 - Packages

In [ ]:
using LinearAlgebra
using SparseArrays
using Printf
using Random
using BenchmarkTools
using TimerOutputs
import MatrixDepot
import Plots

If some packages are missing in your Julia environment, use the following cell to install the specific Julia package

In [ ]:
#remove the comment for the lines below to install Julia package by yourself
#using Pkg
#Pkg.add("PackageName")

In [ ]:
versioninfo()

<a name='2'></a>
## 2 - Test Matrix Collection

[MatrixDepot](https://github.com/JuliaMatrices/MatrixDepot.jl) is a Julia package, which provides an extensible test matrix collection. MatrixDepot provides an easy access to a public matrix collection [SuiteSparse Matrix Collection](https://sparse.tamu.edu/). 

The *SuiteSparse Matrix Collection* is a large and actively growing set of sparse matrices that arise in real applications. The Collection is widely used by the numerical linear algebra community for the development and performance evaluation of sparse matrix algorithms. The matrices in the Collection cover a wide spectrum of domains, including structural engineering, computational fluid dynamics, material science, acoustics, electromagnetic, thermodynamics, computer graphics and vision ......

<center>
<img src="./figs/SuiteSparsePlots.png" alt="centered image">
</center>
<caption><center><font color='purple'><b>Figure 1</b>: Sample Gallery of the SuiteSparse Matrix Collection https://sparse.tamu.edu/about </font></center></caption>

A [search engine](http://yifanhu.net/GALLERY/GRAPHS/search.html) is also available which is able to filter required matrices based different criterion. 

<center>
<img src="./figs/SuiteSparseSearch.png" alt="centered image">
</center>
<caption><center><font color='purple'><b>Figure 2</b>: Search engine for SuiteSparse Matrix Collection http://yifanhu.net/GALLERY/GRAPHS/search.html </font></center></caption>

## Load sparse from SuiteSparse Matrix Collection by MatrixDepot

MatrixDepot parses the name of matrix from SuiteSparse Matrix Collection through a combination of "group name" and "matrix name", e.g., "HB/nos3" refers to matrix "nos3" from the "HB group" (Harwell-Boeing collection).

In [ ]:
#define matrix name in the format "group/matrix"
matname = "HB/nos3"
#load matrix through MatrixDepot
nos3 = MatrixDepot.matrixdepot(matname)
#Get information of loaded matrix
display(nos3)

SuiteSparse Matrix Collection provides also the properties for each matrix, e.g., the number of rows/columns, number of nonzeros, symmetricity, scalar type, condition number, etc. For details about matrix "HB/nos3", please visit: https://www.cise.ufl.edu/research/sparse/matrices/HB/nos3.html

In [ ]:
# Plot the sparsity pattern of loaded matrix
Plots.spy(nos3, legend = :none)

<a name='3'></a>
## 3 - Basic Conjugate Gradient Method

## Implementation of Standard CG

We provides a simple implementation of CG as follows. This implementation uses a Julia package [TimerOutputs](https://github.com/KristofferC/TimerOutputs.jl) for recording the timings the different numerical kernels in CG.

The basic format of timings by **TimerOutputs** is 

either:

```Julia
@timeit Timer "Kernel name" {single line code for this kernel}
```

to record the timings of a single line of code;

or:

```Julia
@timeit Timer "Code block name" begin
    CODE BLOCK
end   
```

to record the timings of a defined code block.

This **timer** is disabled in default. when you first time go through the code, please just ignore these related marcos.


In [ ]:
function CG(A, b, x; maxIter = 5000, tol=1e-10, verbosity = 0, timer = false)
    
    """
    A first implementation of Conjugate Gradient method to solve Ax=b.
    
    Arguments:
    A -- a sparse matrix of size N x N which determines a linear system
    b -- a vector of size N: right-hand side of a linear system
    x -- a vector of size N: initial guess solution of a linear system
    
    Optional:
    maxIter -- maximum iterative steps before the convergence criteria is satisfied, default value is 5000
    tol -- the convergence criteria, default value is 1e-10
    verbosity -- if print the convergence steps (yes, if it is > 0), default value is 0
    timer -- boolean value, indicating if recording the timings, default false
    
    Returns:
    x -- a vector of size N, the approximated solution of linear system
    errors -- a vector of size "step", which includes the convergence errors of solving procedure
    """
    
    Timer = TimerOutput()
    
    if timer
       enable_timer!(Timer) 
    end
    
    @timeit Timer "Total" begin
    
    @timeit Timer "SpMV" r = b - A * x
    @timeit Timer "Copy" p = r
    @timeit Timer "Dot" rsold = r' * r
    step = 0
    errors = zeros(maxIter)
    while step < maxIter
        step += 1  
        @timeit Timer "SpMV" Ap = A * p
        @timeit Timer "Dot" α = rsold / (p' * Ap)
        @timeit Timer "Axpy" x = x + α .* p 
        @timeit Timer "Axpy" r = r - α .* Ap
        @timeit Timer "Dot" rsnew = r' * r
        errors[step] = sqrt(rsnew) 
        if verbosity > 0
            msg = "CG: step: $step, resid: "
            msg *= @sprintf("%.12e\n", errors[step])
            printstyled(msg, bold=false, color=:green)
        end
        
        if sqrt(rsnew) < tol
            break
        end
        @timeit Timer "Axpy" p = r + (rsnew / rsold) * p
        rsold = rsnew
    end 
    end #end Timer "Total"
    
    if timer
        show(Timer)
    end
    return x, errors[1:step]
end

Quesiton:
> Please identify the SpMV operation in the above function.

Answer:
> Put your answer here.

## First try with CG

We have a first try with CG using "HB/nos3" matrix.

In [ ]:
### Get the row and column number of "HB/nos3" matrix
M = size(nos3, 1)
N = size(nos3, 2)
@info "(Row number, column number): " M, N
@info "The scalar type of matrix: " eltype(nos3)

In [ ]:
## Generate a right-hand side in random which constructs a linear system
## together with "HB/nos3" matrix
b = randn(Float64, N)

## Generate a initial guess vector in random
x₀ = randn(Float64, N);

In [ ]:
## Solve defined linear system by CG
x, errors = CG(nos3, b, x₀;verbosity = 1);

In [ ]:
#Plot the convergence curve
Plots.plot(errors, title = "Conjugate Grandient solving linear systems", label = matname, ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
##Compare the solution with the one of a direct solver
x2 = nos3 \ b
@info "Difference between CG solution and Direct solver solution: " maximum(abs.(x.-x2))

Question: 
> What is the condition number of "HB/nos3"?

Answer:
> Write your answer here.

(Tips: the condition number can be found in the page of each matrix on the website of SuiteSparse Matrix Collection. It is denoted as *cond(A)* on the website.)

## Comparison of CG with Sparse Compressed storage format and 2D format

### Benchmark of CG using BenchmarkTools

In [ ]:
### Sparse matrix in CSC format
@benchmark CG(nos3, b, x₀)

In [ ]:
### Sparse matrix in dense 2D format
@benchmark CG(Matrix(nos3), b, x₀)

Question: 
> How much averaged speedup has been obtained from dense matrix format to sparse matrix format on your laptop?

Answer:
> Write your answer here.

### Timings of different numerical kernels in CG

The **timer** is able to be enabled by explicitly set the optional argument `timer=true`.

In [ ]:
### Sparse matrix in CSC format
CG(nos3, b, x₀; timer=true);

In [ ]:
### Sparse matrix in dense 2D format
CG(Matrix(nos3), b, x₀; timer=true);

Question: 
> Compare the two tables of timings above, and describle your observation.

Answer:

> Write your answer here.

<a name='4'></a>
## 4 - Precondtioned Conjugate Gradient Method

Before the introduce of preconditioned Conjugate Gradient method (PCG), let's at first try solving another linear system by the standard CG.

In [ ]:
matname = "HB/bcsstk15"
bcsstk15 = MatrixDepot.matrixdepot(matname)
bcsstk15d = Matrix(bcsstk15)
M = size(bcsstk15, 1)
N = size(bcsstk15, 2);
#matname = "Nasa/nasa4704"

In [ ]:
b = randn(eltype(bcsstk15), N)
x₀ = randn(eltype(bcsstk15), N);
x, errors = CG(bcsstk15, b, x₀;verbosity = 0)
Plots.plot(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":None", ylabel = "residual", xlabel = "iterations", yaxis=:log)

A quick conclusion:
> Standard CG fails to achieve the convergence in 5000 steps for linear system constructed by "HB/bcsstk15" matrix.

Question:
> What is the condition number of "HB/bcsstk15" matrix?

Answer:
> Write your answer here.

## Mathematical idea of PCG

## Implementation of PCG

We provides a simple implementation of PCG as follows:

In [ ]:
function PCG(A, b, x, M; maxIter = 5000, tol=1e-10, verbosity = 0, timer = false)
    
    
    """
    A first implementation of Conjugate Gradient method to solve Ax=b.
    
    Arguments:
    A -- a sparse matrix of size N x N which determines a linear system
    b -- a vector of size N: right-hand side of a linear system
    x -- a vector of size N: initial guess solution of a linear system
    M -- a matrix of size N x N, it is the preconditioning matrix to speedup
         the convergence of CG. It is constructed externally in different manners.
    
    Optional:
    maxIter -- maximum iterative steps before the convergence criteria is satisfied, default value is 5000
    tol -- the convergence criteria, default value is 1e-10
    verbosity -- if print the convergence steps (yes, if it is > 0), default value is 0
    timer -- boolean value, indicating if recording the timings, default false
    
    Returns:
    x -- a vector of size N, the approximated solution of linear system
    errors -- a vector of size "step", which includes the convergence errors of solving procedure
    """
    
    Timer = TimerOutput()
    
    if timer
       enable_timer!(Timer) 
    end
    
    @timeit Timer "Total" begin
        
    @timeit Timer "SpMV" r = b - A * x
    @timeit Timer "MV(Precond)" z = M * r
    @timeit Timer "Copy" p = z
    @timeit Timer "Dot" rsold = r' * z
    step = 0
    errors = zeros(maxIter)
    
    while step < maxIter
        step += 1
        @timeit Timer "SpMV" Ap = A * p
        @timeit Timer "Dot" α = rsold / (p' * Ap)
        @timeit Timer "Axpy" x = x + α * p 
        @timeit Timer "Axpy" r = r - α * Ap
        @timeit Timer "Dot" errors[step] = sqrt(r'*r)
        if verbosity > 0
            msg = "CG: step: $step, resid: "
            msg *= @sprintf("%.12e\n", errors[step])
            printstyled(msg, bold=false, color=:green)
        end
        
        if errors[step] < tol
            break
        end
        @timeit Timer "MV(Precond)" z = M * r
        @timeit Timer "Dot" rsnew = r' * z
        @timeit Timer "Axpy" p = z + (rsnew / rsold) * p
        rsold = rsnew
    end 
        
    end #end Timer "Total"

    if timer
       show(Timer) 
    end
    
    return x, errors[1:step]
end

## Preconditioners
### Jacobi Preconditioner

The Jacobi preconditioner is one of the simplest forms of preconditioning, in which the preconditioner is chosen to be the diagonal of the matrix $P=diag(A)$.  It is efficient for diagonally dominant matrices.

In [ ]:
## Construct the Jacobi precontioning matrix as (diag(A))^{-1}
PJacobi = inv(Diagonal(bcsstk15d))
## Appy the Jacobi preconditioning matrix to PCG
x, errors = PCG(bcsstk15, b, x₀, SparseMatrixCSC(PJacobi); verbosity = 0)
@info "Required iteration number with Jacobi preconditioner is: " size(errors,1)
## Add the convergence curve to plot
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":Jacobi", ylabel = "residual", xlabel = "iterations", yaxis=:log)

### SSOR Preconditioner

In applied mathematics, symmetric successive over-relaxation (SSOR), is a preconditioner. Suppose that an original matrix $A$ is split into diagonal, lower
and upper triangular as $A=D+L+U$, then a SSOR preconditioning matrix can be defined as:

$$P=(D+L)D^{-1}(D+L)^T$$

It can also be parametrised by $\omega$ as:

$$P(\omega)=\frac{\omega}{2-\omega}(\frac{1}{\omega}D+L)D^{-1}(\frac{1}{\omega}D+L)^T$$

A function to construct the SSOR preconitioning matrix is given as follows:

In [ ]:
function SSOR(A; ω=1)    
    
    """
    A first implementation of Conjugate Gradient method to solve Ax=b.
    
    Arguments:
    A -- a sparse matrix of size N x N which determines a linear system
    
    Optional:
    ω -- a scalar: SSOR parameter

    Returns:
    M -- a N x N preconditioning matrix
    """
    
    n = size(A, 1)
    Ad = Matrix(A)
    D = LinearAlgebra.Diagonal(Ad)
    L = LinearAlgebra.LowerTriangular(Ad)

    #M = (D + ω * L) * inv(D) * (D + ω * U)
    M = (ω/(2-ω)) .* ((1/ω) .* D + L) * inv(D) * ((1/ω) .* D + L)'
    return M
end

In [ ]:
## Construct the SSOR precontioning matrix by function SSOR(A; ω=1.8)
ω = 1.8
PSSOR = inv(SSOR(bcsstk15d; ω=ω))
## Appy the SSOR preconditioning matrix to PCG
x, errors = PCG(bcsstk15, b, x₀, PSSOR; verbosity = 0)
@info "Required iteration number with SSOR preconditioner is: " size(errors,1)
## Add the convergence curve to plot
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":SSOR", ylabel = "residual", xlabel = "iterations", yaxis=:log)

### Timings of different preconditioners

In [ ]:
CG(bcsstk15, b, x₀;verbosity = 0, timer=true);

PCG(bcsstk15, b, x₀, SparseMatrixCSC(PJacobi); verbosity = 0, timer=true);

PCG(bcsstk15, b, x₀, PSSOR; verbosity = 0, timer=true);

In [ ]:
@info "Sparsity of Jacobi preconditioning matrix" nnz(SparseMatrixCSC(PJacobi)) / (N * N)
@info "Sparsity of SSOR preconditioning matrix" nnz(SparseMatrixCSC(PSSOR)) / (N * N)

Question:
> Describe your observations by comparing the above three tables

Answer:
> Write your answer here.

(Tips: Considering the sparsity of different preconditioning matrix)

### Condition number of preconditioned linear system

The PCG with precondtioning matrix M is equivalent to applying the conjugate gradient method without preconditioning to the system
$$L^{-1}A(L^{-1})^T\hat{x} = L^{-1}b$$

where 
$$LL^T = M, \quad \hat{x} = E^Tx$$

Thus the condition number of preconditioned linear system is:

$$cond(L^{-1}A(L^{-1})^T)$$

#### Condition number of Jacobi preconditioned linear system

In [ ]:
LJacobi = sqrt.(Diagonal(bcsstk15d))
@info "Condition number of Jacobi preconditioned linear system" cond(inv(LJacobi) * bcsstk15d * (inv(LJacobi))' )

#### Condition number of SSOR preconditioned linear system

In [ ]:
DSSOR = LinearAlgebra.Diagonal(bcsstk15d)
LSSOR = LinearAlgebra.LowerTriangular(bcsstk15d)
LSSOR = sqrt(ω/(2-ω)) .* ((1/ω) .* DSSOR + LSSOR) * sqrt.((inv(DSSOR)))
@info "Condition number of SSOR preconditioned linear system" cond(inv(LSSOR) * bcsstk15d * (inv(LSSOR))' ) 

ToDo: Please fill in the table below with the previously obtained data

| HB/bcsstk15 | None | Jacobi | SSOR |
| :------: | :------: | :------: | :------: |
|Iteration number||||
|Condition number||||
|Total Time(s)||||
|SpMV Time(s)||||
|MV(Precond) ||||

## Play with more matrices

### Matrix: Nasa/nasa4704

In [ ]:
matname = "Nasa/nasa4704"

verb = 0

nasa4704 = MatrixDepot.matrixdepot(matname)
nasa4704d = Matrix(nasa4704)
M = size(nasa4704, 1)
N = size(nasa4704, 2);

b = randn(eltype(nasa4704), N)
x₀ = randn(eltype(nasa4704), N);

#standard CG
x, errors = CG(nasa4704, b, x₀;verbosity = verb)
@info "Required iteration number without preconditioner is: " size(errors,1)
Plots.plot(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":None", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## Construct the Jacobi precontioning matrix as (diag(A))^{-1}
PJacobi = inv(Diagonal(nasa4704d))
## Appy the Jacobi preconditioning matrix to PCG
x, errors = PCG(nasa4704, b, x₀, SparseMatrixCSC(PJacobi); verbosity = verb)
@info "Required iteration number with Jacobi preconditioner is: " size(errors,1)
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":Jacobi", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## Construct the SSOR precontioning matrix by function SSOR(A; ω=1.8)
ω = 1.8
PSSOR = inv(SSOR(nasa4704d; ω=ω))

## Appy the SSOR preconditioning matrix to PCG
x, errors = PCG(nasa4704, b, x₀, PSSOR; verbosity = verb)
@info "Required iteration number with SSOR preconditioner is: " size(errors,1)
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":SSOR", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## recording timings for diferent kernels
CG(nasa4704, b, x₀;verbosity = 0, timer=true);

PCG(nasa4704, b, x₀, SparseMatrixCSC(PJacobi); verbosity = 0, timer=true);

PCG(nasa4704, b, x₀, PSSOR; verbosity = 0, timer=true);

In [ ]:
## Compute condition number of precondtioned linear system
LJacobi = sqrt.(Diagonal(nasa4704d))
@info "Condition number of Jacobi preconditioned linear system" cond(inv(LJacobi) * nasa4704d * (inv(LJacobi))' )

DSSOR = LinearAlgebra.Diagonal(nasa4704d)
LSSOR = LinearAlgebra.LowerTriangular(nasa4704d)
LSSOR = sqrt(ω/(2-ω)) .* ((1/ω) .* DSSOR + LSSOR) * sqrt.((inv(DSSOR)))
@info "Condition number of SSOR preconditioned linear system" cond(inv(LSSOR) * nasa4704d * (inv(LSSOR))' ) 

ToDo: Please fill in the table below with the previously obtained data

| Nasa/nasa4704 | None | Jacobi | SSOR |
| :------: | :------: | :------: | :------: |
|Iteration number||||
|Condition number||||
|Total Time(s)||||
|SpMV Time(s)||||
|MV(Precond) ||||

### Matrix: Boeing/crystm01

In [ ]:
matname = "Boeing/crystm01"

verb = 0

crystm01= MatrixDepot.matrixdepot(matname)
crystm01d = Matrix(crystm01)
M = size(crystm01, 1)
N = size(crystm01, 2);

b = randn(eltype(crystm01), N)
x₀ = randn(eltype(crystm01), N);

In [ ]:
#standard CG
x, errors = CG(crystm01, b, x₀;verbosity = verb)
@info "Required iteration number without preconditioner is: " size(errors,1)
Plots.plot(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":None", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## Construct the Jacobi precontioning matrix as (diag(A))^{-1}
PJacobi = inv(Diagonal(crystm01d))
## Appy the Jacobi preconditioning matrix to PCG
x, errors = PCG(crystm01, b, x₀, PJacobi; verbosity = verb)
@info "Required iteration number with Jacobi preconditioner is: " size(errors,1)
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":Jacobi", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## Construct the SSOR precontioning matrix by function SSOR(A; ω=1.8)
ω = 1.8
PSSOR = inv(SSOR(crystm01d; ω=ω))

## Appy the SSOR preconditioning matrix to PCG
x, errors = PCG(crystm01, b, x₀, PSSOR; verbosity = verb)
@info "Required iteration number with SSOR preconditioner is: " size(errors,1)
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":SSOR", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## recording timings for diferent kernels
CG(crystm01, b, x₀;verbosity = 0, timer=true);

PCG(crystm01, b, x₀, SparseMatrixCSC(PJacobi); verbosity = 0, timer=true);

PCG(crystm01, b, x₀, PSSOR; verbosity = 0, timer=true);

In [ ]:
## Compute condition number of precondtioned linear system
LJacobi = sqrt.(Diagonal(crystm01d))
@info "Condition number of Jacobi preconditioned linear system" cond(inv(LJacobi) * crystm01d * (inv(LJacobi))' )

DSSOR = LinearAlgebra.Diagonal(crystm01d)
LSSOR = LinearAlgebra.LowerTriangular(crystm01d)
LSSOR = sqrt(ω/(2-ω)) .* ((1/ω) .* DSSOR + LSSOR) * sqrt.((inv(DSSOR)))
@info "Condition number of SSOR preconditioned linear system" cond(inv(LSSOR) * crystm01d * (inv(LSSOR))' ) 

ToDo: Please fill in the table below with the previously obtained data

| Boeing/crystm01 | None | Jacobi | SSOR |
| :------: | :------: | :------: | :------: |
|Iteration number||||
|Condition number||||
|Total Time(s)||||
|SpMV Time(s)||||
|MV(Precond) ||||

### Matrix: Bai/mhd3200b

In [ ]:
matname = "Bai/mhd3200b"

verb = 0

mhd3200b = MatrixDepot.matrixdepot(matname)
mhd3200bd = Matrix(mhd3200b)
M = size(mhd3200b, 1)
N = size(mhd3200b, 2);

b = randn(eltype(mhd3200b), N)
x₀ = randn(eltype(mhd3200b), N);

In [ ]:
#standard CG
x, errors = CG(mhd3200b, b, x₀;verbosity = verb)
@info "Required iteration number without preconditioner is: " size(errors,1)
Plots.plot(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":None", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## Construct the Jacobi precontioning matrix as (diag(A))^{-1}
PJacobi = inv(Diagonal(mhd3200bd))
## Appy the Jacobi preconditioning matrix to PCG
x, errors = PCG(mhd3200b, b, x₀, PJacobi; verbosity = verb)
@info "Required iteration number with Jacobi preconditioner is: " size(errors,1)
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":Jacobi", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## Construct the SSOR precontioning matrix by function SSOR(A; ω=1.8)
ω = 1.8
PSSOR = inv(SSOR(mhd3200bd; ω=ω))

## Appy the SSOR preconditioning matrix to PCG
x, errors = PCG(mhd3200b, b, x₀, PSSOR; verbosity = verb)
@info "Required iteration number with SSOR preconditioner is: " size(errors,1)
Plots.plot!(errors, title = "Conjugate Grandient solving linear systems", label = matname * ":SSOR", ylabel = "residual", xlabel = "iterations", yaxis=:log)

In [ ]:
## recording timings for diferent kernels
CG(mhd3200bd, b, x₀;verbosity = 0, timer=true);

PCG(mhd3200bd, b, x₀, SparseMatrixCSC(PJacobi); verbosity = 0, timer=true);

PCG(mhd3200bd, b, x₀, PSSOR; verbosity = 0, timer=true);

In [ ]:
## Compute condition number of precondtioned linear system
LJacobi = sqrt.(Diagonal(mhd3200bd))
@info "Condition number of Jacobi preconditioned linear system" cond(inv(LJacobi) * mhd3200bd * (inv(LJacobi))' )

DSSOR = LinearAlgebra.Diagonal(mhd3200bd)
LSSOR = LinearAlgebra.LowerTriangular(mhd3200bd)
LSSOR = sqrt(ω/(2-ω)) .* ((1/ω) .* DSSOR + LSSOR) * sqrt.((inv(DSSOR)))
@info "Condition number of SSOR preconditioned linear system" cond(inv(LSSOR) * mhd3200bd * (inv(LSSOR))' ) 

ToDo: Please fill in the table below with the previously obtained data

| Bai/mhd3200b | None | Jacobi | SSOR |
| :------: | :------: | :------: | :------: |
|Iteration number||||
|Condition number||||
|Total Time(s)||||
|SpMV Time(s)||||
|MV(Precond) ||||

Question 1:
> Can you explain the relation between the condition number and convergence behaviour?

Answer:
> Write your answer here.

Question 2:
> Please try to summarize the strategy to select different preconditioners (or without preconditioning).

Answer:
> Write your answer here.

(Tips: please consider the tradeoff between convergence behaviour and time-to-solution.)